# Modelo Machine Learning

## Importamos las librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [2]:
reader = Reader()

## Cargamos nuestros datos resultandes del EDA

Los datos puede sacarlos de la carpeta data_para_ML que esta en el drive compartido en el repositorio

In [4]:
df1 = pd.read_csv("C:/Users/Alejandro/Desktop/Proyecto2/Machine_Learning_Data/Calificaciones.csv") # CALIFICACIONES

In [5]:
df2 = pd.read_csv("C:/Users/Alejandro/Desktop/Proyecto2/Machine_Learning_Data/Titulos.csv") # NOMBRES DE PELICULAS

### Seleccionamos la informacion

In [6]:
# Ya que es mucha info la que se esta manejando seleccionare solo una parte de los datos
# Usare los datos del dataframe de calificaciones que exista cada 4 filas
data = Dataset.load_from_df(df1[['userId', 'id', 'score']][::4], reader)

In [7]:
# Separamos nuestros datos
trainset, testset = train_test_split(data, test_size=.25)

In [8]:
# Usaremos un modelo de Singular Value Decomposition
from surprise import SVD
model = SVD()

### Entrenamiento

In [9]:
# Entrenamos el modelo
model.fit(trainset)

### Prediccion

In [10]:
# Predecimos
predictions = model.test(testset)

In [11]:
predictions[1]

Prediction(uid=74275, iid='as3578', r_ui=4.0, est=2.8350031259712742, details={'was_impossible': False})

In [12]:
# Hacemos una predicción al azar para usuario y película
model.predict(500,"as382")

Prediction(uid=500, iid='as382', r_ui=None, est=3.817883911860072, details={'was_impossible': False})

## Eleccion de usuario

In [13]:
# Tomaremos un usuario para hacerle una recomendación
usuario = 200
rating = 4   # Tomamos películas a las que haya calificado con mas de 3 estrellas
df_user = df1[(df1['userId'] == usuario) & (df1['score'] >= rating)]

In [14]:
df_user.head()

,userId,score,id
225004,200,4.0,as545
935886,200,4.0,as2262
2135789,200,4.0,as5161
2404706,200,4.0,as5810
2504650,200,4.5,as6052


In [15]:
df_user = df_user.reset_index(drop=True)

In [16]:
df_user = df_user.merge(df2, on="id")
df_user.head()

,userId,score,id,title
0,200,4.0,as545,the mandela effect
1,200,4.0,as2262,hannie caulder
2,200,4.0,as5161,one step behind the seraphim
3,200,4.0,as5810,lea & pop - baby shark & more kids songs
4,200,4.5,as6052,pinkfong! phonics songs


In [17]:
recomendaciones_usuario = df2.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

(4499, 2)


,id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [18]:
# Debemos extraer las películas que ya ha visto

usuario_vistas = df1[df1['userId'] == usuario]
print(usuario_vistas.shape)
usuario_vistas.head(15)

(67, 3)


,userId,score,id
225004,200,4.0,as545
935886,200,4.0,as2262
2064586,200,2.5,as4989
2135789,200,4.0,as5161
2270885,200,1.0,as5488
2404706,200,4.0,as5810
2504650,200,4.5,as6052
2606521,200,3.0,as6299
2696321,200,4.0,as6517
2758756,200,5.0,as6668


## Peliculas que se recomiendan

In [19]:
# Recomendamos

recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['id'].apply(lambda x: model.predict(usuario, x).est)

In [20]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
recomendaciones_usuario.head()

,id,title,Estimate_Score
4134,as4135,my 3 boys,4.407699
2138,as2139,ian bagg: conversations,4.344689
2167,as2168,humans,4.315795
3972,as3973,fanney khan,4.306397
3531,as3532,15-minute hiit two 8.0 (tabata workout with we...,4.281695


## Evaluacion

In [21]:
from surprise import accuracy

accuracy.rmse(predictions)

RMSE: 0.9953


0.9953459447380018

In [24]:
import gradio as gr
def greet(numero_usuario):
    usuario = int(numero_usuario)
    rating = 4   # Tomamos películas a las que haya calificado con mas de 3 estrellas
    df_user = df1[(df1['userId'] == usuario) & (df1['score'] >= rating)]
    df_user = df_user.reset_index(drop=True)
    df_user = df_user.merge(df2, on="id")
    recomendaciones_usuario = df2.iloc[:4499].copy()
    recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['id'].apply(lambda x: model.predict(usuario, x).est)
    recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
    contar = 0
    lista_recomendaciones = []
    lista_recomendaciones2 = []
    for linea in recomendaciones_usuario["title"]:
        lista_recomendaciones.append(linea)
        break
    for linea2 in recomendaciones_usuario["Estimate_Score"]:
        lista_recomendaciones2.append(linea2)
        return f"Pelicula que te recomiendo: {lista_recomendaciones} creo que la calificarias con un:{lista_recomendaciones2}"

    

demo = gr.Interface(fn=greet, inputs="text", outputs="text",title="Recomendacion de Peliculas por usuario id",
    description="Este es mi modelo de ML para recomendar peliculas",)

demo.launch()

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://8851bc00fc540a2b6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
